In [ ]:
from codeepneat.modules import *
from codeepneat.blueprints import *
from codeepneat.evolution import *

In [ ]:
# SKLEARN
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Load sklearn digits data
digits = load_digits()
X = digits.images  # shape (1797, 8, 8)
y = digits.target  # shape (1797,)

# Normalize pixels to [0,1] float
X = X.astype(np.float32) / 16.0  # original max pixel is 16

# Add channel dim for PyTorch conv: (N, 1, 8, 8)
X = np.expand_dims(X, axis=1)

# Convert to torch tensors
X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y).long()

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y
)

# Create datasets and loaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)


In [6]:
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch

# Transform: convert to tensor and normalize to [0,1]
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to shape (1, 28, 28) and scales to [0,1]
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load MNIST
train_data = MNIST(root="./data", train=True, download=True, transform=transform)
val_data = MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64)

100.0%
100.0%
100.0%
100.0%


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Module pool: same as before but input shape changes
module_pool = {
    0: ModuleGenome("conv", params={"out_channels": 32, "kernel_size": 3}),  # same size out
    1: ModuleGenome("poolconv", params={"out_channels": 64, "kernel_size": 3, "pool_type": "max"}),
    2: ModuleGenome("residual", params={"channels": 64}),  # should internally handle padding too
    3: ModuleGenome("mlp", params={"hidden_dim": 512, "output_dim": 10}),
}

# When assembling network, remember input_shape=(1,8,8) for sklearn digits

best_blueprint, best_model = run_evolution(
    module_pool=module_pool,
    train_loader=train_loader,
    val_loader=val_loader,
    population_size=10,
    generations=3,
    mutation_rate=0.2,
    device=device,
)

print("Best blueprint:", best_blueprint)


Generation 0
Module 0 (residual): C=1, H=28, W=28
Module 1 (residual): C=1, H=28, W=28
Module 2 (mlp): computed input_dim=784 (C=1, H=28, W=28)
Module 2 (mlp): output_dim=10
  Individual 0: fitness=0.9743
Module 0 (conv): C=32, H=28, W=28
Module 1 (residual): C=32, H=28, W=28
Module 2 (residual): C=32, H=28, W=28
Module 3 (mlp): computed input_dim=25088 (C=32, H=28, W=28)
Module 3 (mlp): output_dim=10


KeyboardInterrupt: 

In [ ]:
best_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for xb, yb in val_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = best_model(xb)
        preds = out.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
final_acc = correct / total
print(f"Final validation accuracy of best trained model: {final_acc:.4f}")

In [ ]:
def PGD(model, img, target, c=False, epsilon=3, alpha=0.01, iters=20):
    """
    model: neural network model
    img: original input image (tensor)
    target: ground truth label (tensor)
    epsilon: max perturbation size
    alpha: step size per iteration
    iters: number of iterations
    """
    # Make a copy of the original image to compare for projection
    ori_img = img.clone().detach()
    
    # Initialize perturbed image with the original
    perturbed_img = img.clone().detach().requires_grad_(True)
    
    loss_fn = nn.CrossEntropyLoss()
    
    for i in range(iters):
        # Perform forward pass with the original shape
        outputs = model(perturbed_img)  # Don't flatten, just use original shape

        loss = loss_fn(outputs, target)
        
        model.zero_grad()
        if perturbed_img.grad is not None:
            perturbed_img.grad.data.zero_()
        
        loss.backward()
        
        # Take a step in the direction of gradient sign
        grad_sign = perturbed_img.grad.data.sign()
        perturbed_img = perturbed_img + alpha * grad_sign
        
        # Project back into the epsilon-ball around the original image
        diff = torch.clamp(perturbed_img - ori_img, min=-epsilon, max=epsilon)
        perturbed_img = torch.clamp(ori_img + diff, min=0, max=1).detach()  # Clamp to valid image range
        perturbed_img.requires_grad_()
    
    return perturbed_img.detach()


In [ ]:
best_model.eval()
correct = 0
total = 0

# Set attack parameters for normalized input
epsilon = 0.3
alpha = 0.01
iters = 20

for xb, yb in val_loader:
    xb, yb = xb.to(device), yb.to(device)

    # Run your PGD function
    adv_xb = PGD(best_model, xb, yb, c=False, epsilon=epsilon, alpha=alpha, iters=iters)

    # Predict on adversarial examples
    out = best_model(adv_xb)
    preds = out.argmax(dim=1)
    correct += (preds == yb).sum().item()
    total += yb.size(0)

final_acc = correct / total
print(f"PGD adversarial validation accuracy: {final_acc:.4f}")



In [ ]:
class FFN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8*8, 32, bias=False),
            nn.ReLU(),
            nn.Linear(32, 10, bias=False),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)

        logits = self.linear_relu_stack(x)
        return logits
    
# Training loop

fnn_model = FFN()

epochs = 100

loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(fnn_model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(fnn_model.parameters(), lr=0.1)  # Higher lr to compensate for simplicity
for i in range(epochs):
    for data_batch, label_batch in train_loader:
        output = fnn_model(data_batch)
        loss = loss_fn(output, label_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if i % 100 == 0:
        print(loss.item())

# Evaulate
fnn_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, labels in val_loader:
        data = data
        labels = labels
        
        outputs = fnn_model(data)  # logits
        predicted = torch.argmax(outputs, dim=1)  # class indices

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")